# Correlation or Causation? #

How do we determine if and how two variables (e.g., relative frequency of "sur" and "glad") are related. One classical solution is to compare their variance by asking is it the case that if the first variable change in one direction, does the second variable change in the same or opposite direction? A widely used measure of this similar or opposite variance is Pearson's correlation coefficient *r* (Pearson's *r*), which falls within the range -1 to 1, where *r* = 1 and *r* = -1 are the perfect correlation and anti-correlation respectively.

But what does a correlation mean? We have all heard that *correlation is not causation*, because it is just an association between two variables. This association might be spurious and weak given the variables' domain.

In [ ]:
import os
import pandas as pd

# path to file (change last string for other file)
PATH = os.path.join("..","data","smurf-2018-10-29_13_27.csv")

# import data as DataFrame
df = pd.read_csv(PATH,skiprows=[0,1])

In [ ]:
import numpy as np

seeds = ["sur", "glad"]
x = df.year.values
w = df.total_articles.values.astype("float")

# raw article frequency
y1 = df.sur.values.astype("float")
y2 = df.glad.values.astype("float")

# relative article frequency
y1w = np.divide(y1, w, out=np.zeros_like(y1), where=w!=0)
y2w = np.divide(y2, w, out=np.zeros_like(y2), where=w!=0)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(1, 2, figsize=(9,4), dpi=100)
ax[0].plot(y1w,y2w,"xg")
ax[0].set_xlabel(seeds[0])
ax[0].set_ylabel(seeds[1])


# linear fit data points (first degree polynomial)
p = np.poly1d(np.polyfit(y1w,y2w,1))

ax[1].plot(y1w,y2w,"xg", y1w, p(y1w),'r-',linewidth=3)
ax[1].set_xlabel(seeds[0])


plt.show()

In [ ]:
from scipy.stats import pearsonr

res_corr = pearsonr(y1w,y2w)

print("Pearsons R = {}, p({}) = {}".format(round(res_corr[0],2),len(y1w-2),round(res_corr[1],4)))

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

def granger_cause(response, driver, mxlag = 1):
    """
    test that driver granger causes response
    - H0: driver DOES NOT granger cause response
    - mxlag: maximum lag, function outputs one analysis for each lag
    - NB: remember bidirectional testing for extraneous factors
    """
    df = pd.DataFrame()
    df['E'] = response
    df['D'] = driver
    data = np.asarray(df[['E', 'D']])
    return grangercausalitytests(data, maxlag = mxlag, addconst = True, verbose = True)

res_y2toy1 = granger_cause(y1w,y2w,mxlag = 3)
print("*****")
res_y1toy2 = granger_cause(y2w,y1w,mxlag = 3)